## Projeto: Sistema de Recomendação

## Importação e tratamento dos dados

In [ ]:
# leitura dos arquivos disponíveis no diretório de trabalho

display(dbutils.fs.ls('/FileStore/tables/dados_spotify'))

path,name,size,modificationTime
dbfs:/FileStore/tables/dados_spotify/data.csv,data.csv,29654587,1706802690000
dbfs:/FileStore/tables/dados_spotify/data_by_artist.csv,data_by_artist.csv,4315607,1706802680000
dbfs:/FileStore/tables/dados_spotify/data_by_genres.csv,data_by_genres.csv,576456,1706802681000
dbfs:/FileStore/tables/dados_spotify/data_by_year.csv,data_by_year.csv,21194,1706802682000
dbfs:/FileStore/tables/dados_spotify/data_w_genres.csv,data_w_genres.csv,5224673,1706802686000


In [ ]:
# salvando caminho do arquivo de interesse em uma variável

caminho_data = 'dbfs:/FileStore/tables/dados_spotify/data.csv'

In [ ]:
# salvando os dados no dataframe do spark

df_data = spark.read.csv(caminho_data, inferSchema=True, header=True)

In [ ]:
# visualiando as 5 primeiras linhas de conteúdo do dataframe spark

df_data.show(5)

+-------+----+------------+--------------------+------------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-----------------+
|valence|year|acousticness|             artists|      danceability|duration_ms|energy|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|            tempo|
+-------+----+------------+--------------------+------------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-----------------+
| 0.0594|1921|       0.982|['Sergei Rachmani...|             0.279|     831667| 0.211|       0|4BJqT0PrAfrxzMOxy...|           0.878| 10|   0.665| -20.096|   1|Piano Concerto No...|         4|        1921|     0.0366|           80.954|
|  0.963|1921|       0.732|      ['Dennis Day']|0.819000

In [ ]:
# consultando o tipo de dados do dataframe

type(df_data)

Out[6]: pyspark.sql.dataframe.DataFrame

**Transformando o Dataframe**

In [ ]:
# transformando em um dataframe pandas

df_data = df_data.pandas_api()

In [ ]:
# verificando a transformação no tipo de dados

type(df_data)

Out[8]: pyspark.pandas.frame.DataFrame

In [ ]:
# visualizando os 5 primeiros registros

df_data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.8190000000000001,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.0,7,0.16,-12.441,1,Clancy Lowered the Boom,5,1921,0.415,60.93600000000001
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913,3,0.101,-14.85,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,2.77e-05,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,1.68e-06,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.038,101.665


In [ ]:
# consultando os tipos de cada coluna

df_data.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int32  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  object 
 5   duration_ms       170653 non-null  object 
 6   energy            170653 non-null  object 
 7   explicit          170653 non-null  object 
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  object 
 10  key               170653 non-null  object 
 11  liveness          170653 non-null  object 
 12  loudness          170653 non-null  object 
 13  mode              170653 non-null  object 
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  object 
 16  release_date     

In [ ]:
# separando as variáveis do tipo inteiro e float
colunas_float = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

colunas_int = ['duration_ms', 'mode', 'key', 'explicit', 'popularity']

In [ ]:
# transformando o tipo de dados de cada coluna para o mais adequado
df_data[colunas_float] = df_data[colunas_float].astype(float)
df_data[colunas_int] = df_data[colunas_int].astype(int)

In [ ]:
# consultando tipo das colunas transformadas
df_data.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int32  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170080 non-null  float64
 5   duration_ms       170454 non-null  int64  
 6   energy            170573 non-null  float64
 7   explicit          170606 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170270 non-null  float64
 10  key               170451 non-null  int64  
 11  liveness          170613 non-null  float64
 12  loudness          170621 non-null  float64
 13  mode              170635 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        169496 non-null  int64  
 16  release_date     

In [ ]:
# analisando conteúdo das variáveis do tipo string
df_consulta = df_data[['artists', 'name', 'release_date', 'id']]
df_consulta.head()

,artists,name,release_date,id
0,"['Sergei Rachmaninoff', 'James Levine', 'Berli...","Piano Concerto No. 3 in D Minor, Op. 30: III. ...",1921,4BJqT0PrAfrxzMOxytFOIz
1,['Dennis Day'],Clancy Lowered the Boom,1921,7xPhfUan2yNtyFG0cUWkt8
2,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,Gati Bali,1921,1o6I8BglA6ylDMrIELygv1
3,['Frank Parker'],Danny Boy,1921,3ftBPsC5vPBKxYSee08FDH
4,['Phil Regan'],When Irish Eyes Are Smiling,1921,4d6HGyGT8e121BsdKmw9v6


In [ ]:
# verificando o tipo de dado do primeiro elemento da coluna 'artists'
type(df_data.artists.iloc[0])

Out[15]: str

In [ ]:
# retirando uma amostra dos dados para realizar a transformação de teste.
x = df_data.iloc[0:9]
x

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
5,0.1960,1921,0.579,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.697,395076,0.346,0,4pyw9DVHGStUre4J6hPngr,0.168000,2,0.130,-12.506,1,Gati Mardika,6,1921,0.0700,119.824
6,0.4060,1921,0.996,['John McCormack'],0.518,159507,0.203,0,5uNZnElqOS3W4fRmRYPk4T,0.000000,0,0.115,-10.589,1,The Wearing of the Green,4,1921,0.0615,66.221
7,0.0731,1921,0.993,['Sergei Rachmaninoff'],0.389,218773,0.088,0,02GDntOXexBFUvSgaXLPkd,0.527000,1,0.363,-21.091,0,"Morceaux de fantaisie, Op. 3: No. 2, Prélude i...",2,1921,0.0456,92.867
8,0.7210,1921,0.996,['Ignacio Corsini'],0.485,161520,0.130,0,05xDjWH9ub67nJJk82yfGf,0.151000,5,0.104,-21.508,0,La Mañanita - Remasterizado,0,1921-03-20,0.0483,64.678


In [ ]:
# manipulando string realizando a substituição um caracteres passando uma expressão regular 'REGEX'
x['artists'] = x.artists.str.replace("\[|\]|\'", "")

In [ ]:
# nova manipulação de string, substituindo ',' por ';'
x['artists'] = x.artists.str.replace(",", ";")

In [ ]:
# carregando conjunto de dados e validando as transformações aplicadas
x

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,Sergei Rachmaninoff; James Levine; Berliner Ph...,0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,Dennis Day,0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,Frank Parker,0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,Phil Regan,0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
5,0.1960,1921,0.579,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.697,395076,0.346,0,4pyw9DVHGStUre4J6hPngr,0.168000,2,0.130,-12.506,1,Gati Mardika,6,1921,0.0700,119.824
6,0.4060,1921,0.996,John McCormack,0.518,159507,0.203,0,5uNZnElqOS3W4fRmRYPk4T,0.000000,0,0.115,-10.589,1,The Wearing of the Green,4,1921,0.0615,66.221
7,0.0731,1921,0.993,Sergei Rachmaninoff,0.389,218773,0.088,0,02GDntOXexBFUvSgaXLPkd,0.527000,1,0.363,-21.091,0,"Morceaux de fantaisie, Op. 3: No. 2, Prélude i...",2,1921,0.0456,92.867
8,0.7210,1921,0.996,Ignacio Corsini,0.485,161520,0.130,0,05xDjWH9ub67nJJk82yfGf,0.151000,5,0.104,-21.508,0,La Mañanita - Remasterizado,0,1921-03-20,0.0483,64.678


In [ ]:
# aplicando as transformações no conjunto de dados completo
df_data['artists'] = df_data.artists.str.replace("\[|\]|\'", "")
df_data['artists'] = df_data.artists.str.replace(',', ';')

In [ ]:
# retornando os últimos registros do dataframe e validando as transformações
df_data.tail()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
170648,0.608,2020,0.08460,Anuel AA; Daddy Yankee; KAROL G; Ozuna; J Balvin,0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.734,2020,0.20600,Ashnikko,0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.637,2020,0.10100,MAMAMOO,0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.195,2020,0.00998,Eminem,0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055
170652,0.642,2020,0.13200,KEVVO; J Balvin,0.856,189507,0.721,1,7HmnJHfs0BkFzX4x8j0hkl,0.004710,7,0.1820,-4.928,1,Billetes Azules (with J Balvin),74,2020-10-16,0.1080,94.991


In [ ]:
# consultando conteúdo do diretório
display(dbutils.fs.ls('dbfs:/FileStore/tables'))

path,name,size,modificationTime
dbfs:/FileStore/tables/analise_dados_fies_2021/,analise_dados_fies_2021/,0,0
dbfs:/FileStore/tables/analise_dados_prouni/,analise_dados_prouni/,0,0
dbfs:/FileStore/tables/aula-databricks/,aula-databricks/,0,0
dbfs:/FileStore/tables/dados/,dados/,0,0
dbfs:/FileStore/tables/dados_spotify/,dados_spotify/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/,dados_spotify_tratados/,0,0
dbfs:/FileStore/tables/data.csv,data.csv,2781,1705440822000
dbfs:/FileStore/tables/explorando_tipos_arquivos/,explorando_tipos_arquivos/,0,0


In [ ]:
# criando novo diretório
dbutils.fs.mkdirs('dbfs:/FileStore/tables/dados_spotify_tratados')

Out[22]: True

In [ ]:
# consultando conteúdo do diretório
display(dbutils.fs.ls('dbfs:/FileStore/tables'))

path,name,size,modificationTime
dbfs:/FileStore/tables/analise_dados_fies_2021/,analise_dados_fies_2021/,0,0
dbfs:/FileStore/tables/analise_dados_prouni/,analise_dados_prouni/,0,0
dbfs:/FileStore/tables/aula-databricks/,aula-databricks/,0,0
dbfs:/FileStore/tables/dados/,dados/,0,0
dbfs:/FileStore/tables/dados_spotify/,dados_spotify/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/,dados_spotify_tratados/,0,0
dbfs:/FileStore/tables/data.csv,data.csv,2781,1705440822000
dbfs:/FileStore/tables/explorando_tipos_arquivos/,explorando_tipos_arquivos/,0,0


In [ ]:
# salvando os dados em um arquivo no formato .parquet
df_data.to_parquet('dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet')

In [ ]:
# consultando conteúdo do diretório
display(dbutils.fs.ls('dbfs:/FileStore/tables/dados_spotify_tratados/'))

path,name,size,modificationTime
dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet/,data.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_year/,data_year/,0,0


## Análise de dados

In [ ]:
# importação da biblioteca Pandas do Spark
import pyspark.pandas as ps

In [ ]:
# realizando a leitura do dataframe principal
path = 'dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet/'
df_data = ps.read_parquet(path)
df_data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.917,1970,0.096000,The Velvet Underground,0.624,201440,0.774,0,60ZyiL4lmWzZyGfqyECTqp,0.030900,7,0.0960,-10.391,1,Train Round the Bend - 2015 Remaster,24,1970,0.0315,117.006
1,0.511,1970,0.001900,Ten Years After,0.405,458463,0.543,0,6DYyyUdHzI6RdSx0swUR1i,0.720000,2,0.1860,-9.313,1,Love Like a Man - 2017 Remaster,34,1970-04-01,0.0290,107.598
2,0.466,1970,0.052800,The Mothers Of Invention,0.444,105587,0.568,0,6HJAS8XZO0ctUcN2KsbLRa,0.000010,11,0.5120,-8.800,0,Oh No,24,1970-08-10,0.0327,124.319
3,0.523,1970,0.081100,Three Dog Night,0.502,174707,0.669,0,7sZ74qmKb1nyGKUgHROJ1n,0.000945,7,0.0906,-11.725,1,One Man Band,19,1970-01-01,0.0912,121.089
4,0.501,1970,0.000128,The Rolling Stones,0.273,246413,0.866,0,095WtNlSHE8TMB2gQ1fdTx,0.790000,11,0.9610,-7.598,1,Street Fighting Man - Live,25,1970-09-04,0.0347,134.891


In [ ]:
# analisando dados estatísticos iniciais das variáveis numéricas
df_data.describe()

,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo
count,170653.000000,170653.000000,170653.000000,170080.000000,1.704540e+05,1.705730e+05,170606.000000,170270.000000,170451.000000,170613.000000,170621.000000,170635.000000,169496.000000,170042.000000,170404.000000
mean,0.528587,1976.787241,0.502115,0.537640,2.304044e+05,5.332695e+02,161.708058,36.464539,20.310635,3.575240,-11.417018,7.382782,31.543547,5.064387,118.226091
std,0.263171,25.917853,0.376032,0.175959,1.263665e+05,1.350096e+04,6984.208830,3139.122132,2119.315151,981.333683,5.729827,1867.348785,21.803031,98.121279,67.002730
min,0.000000,1921.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-60.000000,-36.000000,-39.000000,-30.073000,-8.331000
25%,0.317000,1956.000000,0.101000,0.416000,1.695350e+05,2.560000e-01,0.000000,0.000000,2.000000,0.098700,-14.584000,0.000000,12.000000,0.034900,93.206000
50%,0.539000,1977.000000,0.516000,0.548000,2.072000e+05,4.730000e-01,0.000000,0.000215,5.000000,0.136000,-10.549000,1.000000,34.000000,0.045000,114.636000
75%,0.747000,1999.000000,0.893000,0.668000,2.621550e+05,7.050000e-01,0.000000,0.102000,8.000000,0.262000,-7.146000,1.000000,48.000000,0.076200,135.552000
max,1.000000,2020.000000,0.996000,0.988000,5.403500e+06,1.622000e+06,693933.000000,475880.000000,503320.000000,290627.000000,11.000000,701187.000000,100.000000,2009.000000,2010.000000


**Explorando a coluna Year**

In [ ]:
# analisando os dados únicos de anos
df_data.year.unique()

Out[31]: 0     1959
1     1990
2     1975
3     1977
4     2003
5     2007
6     2018
7     1974
8     2015
9     1955
10    2006
11    1978
12    1961
13    2013
14    1952
15    1956
16    1988
17    1997
18    1994
19    1968
20    2014
21    1951
22    1930
23    1973
24    1979
25    2019
26    1971
27    1950
28    1949
29    1966
30    2004
31    1991
32    1967
33    1969
34    1982
35    1945
36    1957
37    1989
38    1996
39    1998
40    1963
41    1965
42    1985
43    2020
44    1960
45    1970
46    2012
47    1987
48    2009
49    2016
50    1995
51    1980
52    1953
53    2001
54    1972
55    1992
56    1958
57    1983
58    2005
59    1954
60    1984
61    2000
62    1964
63    1981
64    2010
65    1986
66    2011
67    1976
68    2008
69    1940
70    2017
71    1999
72    1948
73    1962
74    2002
75    1993
76    1942
77    1946
78    1947
79    1926
80    1935
81    1928
82    1927
83    1939
84    1944
85    1938
86    1943
87    1941
88    1936
89    1931
9

In [ ]:
# realizando a contagem de qtde de anos únicos
len(df_data.year.unique())

Out[34]: 100

In [ ]:
# analisando a contagem de registros por ano
df_data['year'].value_counts().sort_index()

Out[36]: 1921     150
1922      71
1923     185
1924     236
1925     278
1926    1378
1927     615
1928    1261
1929     952
1930    1924
1931     966
1932     502
1933     691
1934     578
1935    1518
1936    1100
1937     657
1938     613
1939    1000
1940    2000
1941     960
1942    1682
1943     710
1944     752
1945    2000
1946    1724
1947    1650
1948    1900
1949    2000
1950    2000
1951    2000
1952    2000
1953    1950
1954    2000
1955    2000
1956    2000
1957    1950
1958    2000
1959    1950
1960    1850
1961    1900
1962    1900
1963    2000
1964    1950
1965    1950
1966    2000
1967    1999
1968    2000
1969    2000
1970    2000
1971    2000
1972    2000
1973    2000
1974    2000
1975    2000
1976    2000
1977    2000
1978    2000
1979    2000
1980    1950
1981    2000
1982    2000
1983    2000
1984    2000
1985    2000
1986    2000
1987    1950
1988    1950
1989    2000
1990    2000
1991    1950
1992    2000
1993    2000
1994    2000
1995    2000
1996    2000
199

In [ ]:
# analisando as informações por meio de um gráfico
df_data['year'].value_counts().sort_index().plot.bar()

In [ ]:
# agrupando as músicas por décadas
df_data['decade'] = df_data.year.apply(lambda year: f'{(year//10)*10}s')

In [ ]:
# verificando base com a criação da nova coluna
df_data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,decade
0,0.917,1970,0.096000,The Velvet Underground,0.624,201440,0.774,0,60ZyiL4lmWzZyGfqyECTqp,0.030900,7,0.0960,-10.391,1,Train Round the Bend - 2015 Remaster,24,1970,0.0315,117.006,1970s
1,0.511,1970,0.001900,Ten Years After,0.405,458463,0.543,0,6DYyyUdHzI6RdSx0swUR1i,0.720000,2,0.1860,-9.313,1,Love Like a Man - 2017 Remaster,34,1970-04-01,0.0290,107.598,1970s
2,0.466,1970,0.052800,The Mothers Of Invention,0.444,105587,0.568,0,6HJAS8XZO0ctUcN2KsbLRa,0.000010,11,0.5120,-8.800,0,Oh No,24,1970-08-10,0.0327,124.319,1970s
3,0.523,1970,0.081100,Three Dog Night,0.502,174707,0.669,0,7sZ74qmKb1nyGKUgHROJ1n,0.000945,7,0.0906,-11.725,1,One Man Band,19,1970-01-01,0.0912,121.089,1970s
4,0.501,1970,0.000128,The Rolling Stones,0.273,246413,0.866,0,095WtNlSHE8TMB2gQ1fdTx,0.790000,11,0.9610,-7.598,1,Street Fighting Man - Live,25,1970-09-04,0.0347,134.891,1970s


In [ ]:
# realizando uma cópia do dataframe mas com colunas específicas
df_data_2 = df_data[['decade']]
df_data_2['qtde'] = 1

In [ ]:
# consultando info do novo dataframe
df_data_2.head()

,decade,qtde
0,1970s,1
1,1970s,1
2,1970s,1
3,1970s,1
4,1970s,1


In [ ]:
# criando novo dataframe agregando os dados pela coluna 'decade' e aplicando a soma de musicas
df_data_2 = df_data_2.groupby('decade').sum()
df_data_2

,qtde
decade,
1970s,20000
2000s,19646
1980s,19850
1960s,19549
1930s,9549
2010s,19774
2020s,2030
1990s,19901
1950s,19850


In [ ]:
# classicando os dados em ordem crescente por decada
df_data_2.sort_index()

,qtde
decade,
1920s,5126
1930s,9549
1940s,15378
1950s,19850
1960s,19549
1970s,20000
1980s,19850
1990s,19901
2000s,19646


In [ ]:
# visualização gráfica dos dados
df_data_2.sort_index().plot.bar(y='qtde')

In [ ]:
# consultando conteúdo do diretório
display(dbutils.fs.ls('dbfs:/FileStore/tables/dados_spotify_tratados/'))

path,name,size,modificationTime
dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet/,data.parquet/,0,0
dbfs:/FileStore/tables/dados_spotify_tratados/data_year.parquet/,data_year.parquet/,0,0


**Analisando novo conjunto de dados 'data_year'**

In [ ]:
# realizando a leitura do arquivo '.parquet'
path = 'dbfs:/FileStore/tables/dados_spotify_tratados/data_year.parquet'
df_year = ps.read_parquet(path)
df_year.head()

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5


In [ ]:
# verificando a qtde de anos únicos
len(df_year.year.unique())

Out[4]: 100

In [ ]:
# criando gráfico de linha e analisando a relação entre o ano e o tempo de duração das músicas
df_year.plot.line(x='year', y='duration_ms')

In [ ]:
# criando nova visualização analisando a relação do ano com outras características da música
df_year.plot.line(x='year', y=['acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'speechiness', 'valence'])

In [ ]:
# agrupando as músicas por décadas
df_year['decade'] = df_year.year.apply(lambda year: f'{(year//10)*10}s')

In [ ]:
# criando nova variável no dataframe
df_year['qtde'] = 1

In [ ]:
# consultando últimos registros
df_year.tail()

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,decade,qtde
95,1,2016,0.284171,0.600202,221396.510295,0.592855,0.093984,0.181170,-8.061056,0.104313,118.652630,0.431532,59.647190,0,2010s,1
96,1,2017,0.286099,0.612217,211115.696787,0.590421,0.097091,0.191713,-8.312630,0.110536,117.202740,0.416476,63.263554,1,2010s,1
97,1,2018,0.267633,0.663500,206001.007133,0.602435,0.054217,0.176326,-7.168785,0.127176,121.922308,0.447921,63.296243,1,2010s,1
98,1,2019,0.278299,0.644814,201024.788096,0.593224,0.077640,0.172616,-7.722192,0.121043,120.235644,0.458818,65.256542,1,2010s,1
99,1,2020,0.219931,0.692904,193728.397537,0.631232,0.016376,0.178535,-6.595067,0.141384,124.283129,0.501048,64.301970,1,2020s,1


In [ ]:
# criando uma cópia do dataframe com colunas específicas
df_year_2 = df_year[['decade']]
df_year_2['qtde'] = 1

In [ ]:
# analisando novo dataframe
df_year_2.head()

,qtde
decade,
1970s,10
2000s,10
1980s,10
1960s,10
1930s,10


In [ ]:
# criando novo dataframe agregando os dados pela coluna 'decade' e aplicando a soma de músicas
df_year_2 = df_year_2.groupby('decade').sum()
df_year_2


,qtde
decade,
1970s,10
2000s,10
1980s,10
1960s,10
1930s,10
1920s,9
2010s,10
2020s,1
1990s,10


In [ ]:
# agregando as características das músicas por década e calculando a média de cada uma
df_year_3 = df_year.groupby('decade').agg({
    'acousticness': 'mean',
    'danceability': 'mean',
    'duration_ms': 'mean',
    'energy': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean', 
    'tempo': 'mean',
    'valence': 'mean', 
    'popularity': 'mean',
    'qtde': 'sum' }).sort_index().reset_index()
df_year_3

,decade,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,qtde
0,1920s,0.869176,0.559094,189520.609165,0.253390,0.398363,0.217654,-16.163132,0.192833,110.539268,0.561653,1.504310,9
1,1930s,0.874072,0.541888,208295.762372,0.284158,0.272649,0.225668,-14.190930,0.184916,112.254898,0.564600,2.777264,10
2,1940s,0.877998,0.473281,221154.015009,0.256639,0.377673,0.222325,-15.196490,0.145802,107.194472,0.490011,1.865367,10
3,1950s,0.840190,0.476839,220517.635873,0.286783,0.247689,0.209141,-14.730952,0.093532,110.976998,0.478554,10.722800,10
4,1960s,0.626345,0.494642,211668.931336,0.413415,0.158230,0.208682,-12.694151,0.057821,115.132553,0.551599,26.446169,10
5,1970s,0.400162,0.524927,254051.705500,0.533716,0.116031,0.216406,-11.424966,0.059764,119.858574,0.585098,35.055800,10
6,1980s,0.298690,0.546382,252124.016915,0.594663,0.122222,0.204638,-11.227398,0.062028,121.344803,0.564402,37.530731,10
7,1990s,0.307508,0.566096,248580.479991,0.586126,0.109719,0.196551,-10.002535,0.080539,119.404388,0.545034,44.193423,10
8,2000s,0.269677,0.574115,239515.479891,0.651675,0.083780,0.195598,-7.499272,0.087720,121.268727,0.530241,49.740330,10
9,2010s,0.264278,0.597184,227118.107452,0.628704,0.087661,0.189488,-7.517522,0.098801,120.807677,0.456069,57.644690,10


In [ ]:
# analisando graficamente como as características das músicas variam por década
df_year_3.plot.line(x='decade', y=['acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'speechiness', 'valence'])